# Classifier

## Google COLAB Settings
In this section are certain processes that should be run when running the code through Google COLAB so to have access to a GPU. If such is the case, uncomment the sections and run them sequentially. otherwise, feel free to skip directly to [Imports](#Imports).

### Installs

In [ ]:
# %%capture
# from os.path import exists
# from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
# platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
# cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
# accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
# !pip install livelossplot

### Google Drive
This portion is exclusively for development on _my_ end. I use Google Drive to access the training/testing data without having to redownload it each time the Google COLAB runtime is reset. 

Of course anyone who does not have access to my Google credentials will not be able to access my Drive. As such, these users should skip directly to [Imports](#Imports). The result will be that torchvision will personally download the CIFAR data from the web each time the COLAB runtime is reset.

#### Mounting Drive
This mounts Google Drive to the local runtime. If Drive is already mounted, then of course, it will not try to mount it again. It will of course ask for authentication.

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

#### Importing data from Drive
Here I import the CIFAR data which I have previously downloaded and stored in my Google Drive. To do so I copy the corresponding directory from my Google Drive into the COLAB Runtime to avoid having to redownload it each time my COLAB runtime is reset. The ```-n``` flag is set to avoid overwriting.

In [ ]:
# !cp -r -n /content/gdrive/My\ Drive/Education/Undergraduate/Year_3/Computer_Science/SSA/Machine_Learning/Coursework/ML_Classifier-Pegasus-Generator/data /content/

## Imports

In [ ]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision  # provides specific datasets
import matplotlib.pyplot as plt  # provides plotting capabilities
from livelossplot import PlotLosses  # provides live plotting capabilities

## PyTorch settings

In [ ]:
# sets the device for the user.
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device being used:", device)

## Functions

In [ ]:
def cycle(iterable):
    """Helper function to make getting another batch of data easier"""
    while True:
        for x in iterable:
            yield x

            
def plot_image(i, predictions_array, true_label, img):
    """Plots predicted images"""
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    color = '#335599' if predicted_label == true_label else '#ee4433'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                  100*np.max(predictions_array),
                                  class_names[true_label]),
                                  color=color)

    
def plot_value_array(i, predictions_array, true_label):
    """Plots the value arrays associated with particular predictions"""
    predictions_array, true_label = predictions_array[i], true_label[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    thisplot = plt.bar(range(len(class_names)), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('#ee4433')
    thisplot[true_label].set_color('#335599')

## Classes

In [ ]:
'''A simple classifier'''
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        # initialize network layers
        layers = nn.ModuleList()
        # 3 (colors) by 32 (width) by 32 (height) tensor 
        layers.append(nn.Linear(in_features=3 * 32 * 32, out_features=512))
        # rectifier layer
        layers.append(nn.ReLU())
        # 100 class outputs
        layers.append(nn.Linear(in_features=512, out_features=100))
        self.layers = layers

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten input as we're using linear layers
        for m in self.layers:
            x = m(x)
        return x

## Dataset Setup

In [ ]:
# define list containing class names
class_names = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'computer_keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle',
               'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm', ]

# load CIFAR100 training data, shuffling at each epoch, with a batch size of 16
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR100('data', train=True, download=True, transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])),
    shuffle=True, batch_size=16, drop_last=True)

# does the same but for testing data, with no shuffling
test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR100('data', train=False, download=True, transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])),
    shuffle=False, batch_size=16, drop_last=True)

# create iterators for training and testing
train_iterator = iter(cycle(train_loader))
test_iterator = iter(cycle(test_loader))

print(f'> Size of training dataset {len(train_loader.dataset)}')
print(f'> Size of test dataset {len(test_loader.dataset)}')

## Viewing Some of Test Dataset

In [ ]:
# create pyplot figure
plt.figure(figsize=(10, 10))
# make a 5 by 5 grid of images from test dataset
for i in range(25):
    plt.subplot(5, 5, i + 1)
    # get rid of tickmarks
    plt.xticks([])
    plt.yticks([])
    # get rid of grid
    plt.grid(False)
    # show the image
    plt.imshow(test_loader.dataset[i][0].permute(
        0, 2, 1).contiguous().permute(2, 1, 0), cmap=plt.cm.binary)
    # label
    plt.xlabel(class_names[test_loader.dataset[i][1]])

## Training/Testing Set up

In [ ]:
# initialize model object
N = MyNetwork().to(device)

print(f'> Number of network parameters {len(torch.nn.utils.parameters_to_vector(N.parameters()))}')

# initialise the optimiser (stochastic gradient descent)
optimiser = torch.optim.SGD(N.parameters(), lr=0.001)
# set start epoch
epoch = 0
# initialize live loss plot object
liveplot = PlotLosses()

## Training/Testing

In [ ]:
while (epoch < 100): # training for 100 epochs

    # arrays for metrics
    logs = {}
    train_loss_arr = np.zeros(0)
    train_acc_arr = np.zeros(0)
    test_loss_arr = np.zeros(0)
    test_acc_arr = np.zeros(0)

    # iterate over some of the train dateset
    for i in range(1000):
        # get input and respective targets from training dataset
        x, t = next(train_iterator)
        x, t = x.to(device), t.to(device)
        
        # initialize the gradient to zero
        optimiser.zero_grad()
        # calculate prediction by running input through Neural Network
        p = N(x)
        pred = p.argmax(dim=1, keepdim=True)
        
        # calculate loss between prediction and target
        loss = torch.nn.functional.cross_entropy(p, t)
        # backpropagate loss
        loss.backward()
        # performs a parameter update
        optimiser.step()
    
        # record the loss for this epoch
        train_loss_arr = np.append(train_loss_arr, loss.cpu().data)
        # record the accuracy for this epoch
        train_acc_arr = np.append(train_acc_arr, pred.data.eq(
            t.view_as(pred)).float().mean().item())

    # iterate entire test dataset
    for x, t in test_loader:
        # get input and respective targets from testing dataset
        x, t = x.to(device), t.to(device)
        
        # calculate prediction by running input through Neural Network
        p = N(x)
        # calculate loss
        loss = torch.nn.functional.cross_entropy(p, t)
        pred = p.argmax(dim=1, keepdim=True)

        # record the loss for this epoch
        test_loss_arr = np.append(test_loss_arr, loss.cpu().data)
        # record the accuracy for this epoch
        test_acc_arr = np.append(test_acc_arr, pred.data.eq(
            t.view_as(pred)).float().mean().item())
    
    # draw the training results live
    # NOTE: live plot library has dumb naming forcing our 'test' to be called 'validation'
    liveplot.update({
        'accuracy': train_acc_arr.mean(),
        'val_accuracy': test_acc_arr.mean(),
        'loss': train_loss_arr.mean(),
        'val_loss': test_loss_arr.mean()
    })
    liveplot.draw()
    
    # move on to the next epoch
    epoch = epoch + 1

In [ ]:
print ("Training completed")

## Inference

In [ ]:
test_images, test_labels = next(test_iterator)
test_images, test_labels = test_images.to(device), test_labels.to(device)
# perform inference on the test dataset - use softmax to format this as a sum of probabilities normalized to 1
test_preds = torch.softmax(N(test_images).view(test_images.size(0), len(class_names)), dim=1).data.squeeze().cpu().numpy()


## Plotting Results

In [ ]:
num_rows = 4
num_cols = 4
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, test_preds, test_labels.cpu(), test_images.cpu().squeeze().permute(1,3,2,0).contiguous().permute(3,2,1,0))
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, test_preds, test_labels)